In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_619698/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 10:27:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/16 10:28:01 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Friendship table
friendship_data = {
    'user1_id': [1, 1, 1, 2, 2, 2, 6],
    'user2_id': [2, 3, 4, 3, 4, 5, 1]
}

# Sample data for the Likes table
likes_data = {
    'user_id': [1, 2, 3, 4, 5, 6, 2, 3, 6],
    'page_id': [88, 23, 24, 56, 11, 33, 77, 77, 88]
}

# Convert data to DataFrames
friendship_df = pd.DataFrame(friendship_data)
likes_df = pd.DataFrame(likes_data)

# Display the DataFrames
print("Friendship table:")
print(friendship_df)
print("\nLikes table:")
print(likes_df)


df_person = spark.createDataFrame(friendship_df)
df_person.createOrReplaceTempView("Friendship")

df_person = spark.createDataFrame(likes_df)
df_person.createOrReplaceTempView("Likes")



Friendship table:
   user1_id  user2_id
0         1         2
1         1         3
2         1         4
3         2         3
4         2         4
5         2         5
6         6         1

Likes table:
   user_id  page_id
0        1       88
1        2       23
2        3       24
3        4       56
4        5       11
5        6       33
6        2       77
7        3       77
8        6       88


In [26]:
query = """
    with user as (
        select
            distinct
            user_id,
            friend_id
        from (
            select user1_id as user_id, user2_id as friend_id from Friendship
            union all
            select user2_id as user_id, user1_id as friend_id from Friendship
        )
        order by user_id
    ),
    
    friend_like as (
        select 
            u.user_id,
            u.friend_id,
            l1.page_id as friend_like_page
        from user u 
            left join Likes l1 on u.friend_id = l1.user_id
        order by user_id, friend_id
    ),
    
    user_like as (
        select 
            u.user_id,
            l2.page_id as user_like_page
        from user u 
            left join Likes l2 on u.user_id = l2.user_id
        order by user_id
    )
    
    select
        user_id,
        friend_like_page as page_id,
        count(distinct friend_id) as friends_likes
    from (
        select 
            user_id,
            friend_id,
            friend_like_page
        from friend_like f 
        where (f.user_id, f.friend_like_page) not in (
            select user_id, user_like_page from user_like
        )
    )
    group by user_id, friend_like_page
    order by user_id
"""

In [27]:
spark.sql(query).show(50)

+-------+-------+-------------+
|user_id|page_id|friends_likes|
+-------+-------+-------------+
|      1|     33|            1|
|      1|     56|            1|
|      1|     23|            1|
|      1|     77|            2|
|      1|     24|            1|
|      2|     88|            1|
|      2|     56|            1|
|      2|     11|            1|
|      2|     24|            1|
|      3|     88|            1|
|      3|     23|            1|
|      4|     88|            1|
|      4|     77|            1|
|      4|     23|            1|
|      5|     77|            1|
|      5|     23|            1|
+-------+-------+-------------+

